In [1]:
import pandas as pd
import numpy as np
import os 
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random
import time

In [2]:
data_file = "output2.txt"
output_dir = Path(f"./output_dir/")

In [3]:
loader = TextLoader(data_file, encoding='utf-8')
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap = 100,
    length_function = len,
    is_separator_regex=False
)
pages = splitter.split_documents(documents)
pages = pages[10000:10010]
print(f"Number of pages: {len(pages)}")
print(pages[3].page_content[:100])

Number of pages: 10
Cdb = AllocateZeroPool (EFI_SCSI_OP_LENGTH_SIXTEEN);
  if (Cdb == NULL) {
    Status = EFI_OUT_OF_RE


In [4]:
import httpx
api_key = os.getenv('API_KEY')
base_url = os.getenv('API_URL')
max_output_tokens = 300
streaming = False
http_client = httpx.Client(verify=False)
available_models = [
    "mixtral-8x7b-instruct-v01", 
    "gemma-7b-it", 
    "mistral-7b-instruct-v02", 
    "llama-2-70b-chat", 
    "phi-3-mini-128k-instruct", 
    "llama-3-8b-instruct"]

In [5]:
import sys
import json
from yachalk import chalk
from langchain_openai import ChatOpenAI,OpenAI

# Append the parent directory to the system path
sys.path.append("..")

# Initialize the ChatOpenAI client
client = OpenAI(
    base_url=base_url,
    model=available_models[0],
    http_client=http_client,
    api_key=api_key
)

def trim_incomplete_json(json_string):
    # Find the last occurrence of '}]' or '},' in the string
    last_complete = max(json_string.rfind('}]'), json_string.rfind('},'))
    
    if last_complete != -1:
        # If found, trim the string to that point and add closing bracket if needed
        trimmed = json_string[:last_complete+1]
        if not trimmed.endswith(']'):
            trimmed += ']'
        return trimmed
    else:
        # If no complete object found, return empty list
        return '[]'

def extract_concepts(prompt: str, metadata: dict = {}) -> list:
    SYS_PROMPT = (
        "Your task is to extract the key concepts (and non-personal entities) mentioned in the given context. "
        "Extract only the most important and atomistic concepts, breaking them down into simpler concepts if needed. "
        "Categorize the concepts into one of the following categories: "
        "[import, concept, function, object, document, class, condition, misc].\n"
        "Format your output as a list of JSON objects in the following format:\n"
        "[\n"
        "   {\n"
        '       "entity": "The Concept",\n'
        '       "importance": "The contextual importance of the concept on a scale of 1 to 5 (5 being the highest)",\n'
        '       "category": "The Type of Concept"\n'
        "   },\n"
        "   {...}\n"
        "]"
    )

    messages = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": prompt}
    ]
    
    response = client.invoke(input=messages)
    print("Extract Prompt ", response)
    try:
        result = json.loads(response)
    except json.JSONDecodeError:
        print("\n\nWARNING ### Incomplete JSON detected. Attempting to trim...")
        trimmed_response = trim_incomplete_json(response)
        print(trimmed_response+"\n#####################################################################################################")
        try:
            result = json.loads(trimmed_response)
        except json.JSONDecodeError:
            print("\n\nERROR ### Failed to parse even after trimming. Here is the buggy response: ", response, "\n\n")
            return None

    if result is not None:
        result = [dict(item, **metadata) for item in result]

    return result

def graph_prompt(input_text: str, metadata: dict = {}) -> list:
    SYS_PROMPT = (
        "You are a network graph maker who extracts terms and their relations from a given context. "
        "You are provided with a context chunk (delimited by ```). Your task is to extract the ontology "
        "of terms mentioned in the given context. These terms should represent the key concepts according to the context.\n"
        "Thought 1: While traversing through each sentence, think about the key terms mentioned in it.\n"
        "\tTerms may include object, entity, class, import, function, \n"
        "\tcondition, parameters, documents, service, concept, etc.\n"
        "\tTerms should be as atomistic as possible.\n\n"
        "Thought 2: Think about how these terms can have one-on-one relations with other terms.\n"
        "\tTerms mentioned in the same code or file are typically related to each other.\n"
        "\tTerms can be related to many other terms.\n\n"
        "Thought 3: Determine the relation between each related pair of terms.\n\n"
        "Format your output as a list of JSON objects. Each element of the list contains a pair of terms do not provide an explanation, JUST THE JSON OUTPUT "
        "and the relationship between them, as follows:\n"
        "[\n"
        "   {\n"
        '       "node_1": "A concept from the extracted ontology",\n'
        '       "node_2": "A related concept from the extracted ontology",\n'
        '       "edge": "The relationship between node_1 and node_2 in one or two sentences"\n'
        "   },\n"
        "   {...}\n"
        "]"
    )

    USER_PROMPT = f"context: ```{input_text}``` \n\n output: "

    messages = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": USER_PROMPT}
    ]

    response = client.invoke(input=messages)
    # print("Graph Prompt ", response)
    try:
        result = json.loads(response)
    except json.JSONDecodeError:
        # print("\n\nWARNING ### Incomplete JSON detected. Attempting to trim...")
        trimmed_response = trim_incomplete_json(response)
        # print(trimmed_response)
        # print("################################################################################################################")
        try:
            result = json.loads(trimmed_response)
        except json.JSONDecodeError:
            print("\n\nERROR ### Failed to parse even after trimming. Here is the buggy response: ")
            return None

    if result is not None:
        result = [dict(item, **metadata) for item in result]

    return result

In [6]:
import uuid
import pandas as pd
import numpy as np


def documents2Dataframe(documents) -> pd.DataFrame:
    rows = []
    for chunk in documents:
        row = {
            "text": chunk.page_content,
            **chunk.metadata,
            "chunk_id": uuid.uuid4().hex,
        }
        rows = rows + [row]

    df = pd.DataFrame(rows)
    return df


def df2ConceptsList(dataframe: pd.DataFrame) -> list:
    # dataframe.reset_index(inplace=True)
    results = dataframe.apply(
        lambda row: extract_concepts(
            row.text, {"chunk_id": row.chunk_id, "type": "concept"}
        ),
        axis=1,
    )
    # invalid json results in NaN
    results = results.dropna()
    results = results.reset_index(drop=True)

    ## Flatten the list of lists to one single list of entities.
    concept_list = np.concatenate(results).ravel().tolist()
    return concept_list


def concepts2Df(concepts_list) -> pd.DataFrame:
    ## Remove all NaN entities
    concepts_dataframe = pd.DataFrame(concepts_list).replace(" ", np.nan)
    concepts_dataframe = concepts_dataframe.dropna(subset=["entity"])
    concepts_dataframe["entity"] = concepts_dataframe["entity"].apply(
        lambda x: x.lower()
    )

    return concepts_dataframe


def df2Graph(dataframe: pd.DataFrame, model=None) -> list:
    total_rows = len(dataframe)
    processed_rows = 0
    start_time = time.time()

    def process_row(row):
        nonlocal processed_rows
        result = graph_prompt(row.text, {"chunk_id": row.chunk_id})
        processed_rows += 1
        elapsed_time = time.time() - start_time
        avg_time_per_row = elapsed_time / processed_rows
        estimated_time_remaining = (total_rows - processed_rows) * avg_time_per_row

        print(f"\rProcessing: {processed_rows}/{total_rows} rows | "
              f"Elapsed: {elapsed_time:.2f}s | "
              f"Estimated time remaining: {estimated_time_remaining:.2f}s", 
              end="", flush=True)
        return result

    results = dataframe.apply(process_row, axis=1)

    print("\nProcessing complete!")
    print(results)

    # Filter out None values and flatten the list of lists to one single list of entities.
    concept_list = [item for sublist in results if sublist is not None for item in sublist]
    return concept_list

def graph2Df(nodes_list) -> pd.DataFrame:
    ## Remove all NaN entities
    graph_dataframe = pd.DataFrame(nodes_list).replace(" ", np.nan)
    graph_dataframe = graph_dataframe.dropna(subset=["node_1", "node_2"])
    graph_dataframe["node_1"] = graph_dataframe["node_1"].apply(lambda x: x.lower())
    graph_dataframe["node_2"] = graph_dataframe["node_2"].apply(lambda x: x.lower())

    return graph_dataframe

In [7]:

df = documents2Dataframe(pages)
print(len(df))

10


In [8]:
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df)
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    dfg1.to_csv(output_dir/"graph.csv", sep="|", index=False)
    df.to_csv(output_dir/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(output_dir/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)

Processing: 10/10 rows | Elapsed: 104.40s | Estimated time remaining: 0.00s
Processing complete!
0    [{'node_1': 'Cdb', 'node_2': 'AllocateZeroPool...
1    [{'node_1': 'ScsiIo', 'node_2': 'SenseData', '...
2    [{'node_1': 'EFI_SCSI_IO_PROTOCOL', 'node_2': ...
3    [{'node_1': 'Cdb', 'node_2': 'The context of t...
4    [{'node_1': 'ScsiIo', 'node_2': 'SCSI IO Proto...
5    [{'node_1': 'EFI_SCSI_IO_PROTOCOL', 'node_2': ...
6    [{'node_1': 'Cdb', 'node_2': 'AllocateZeroPool...
7    [{'node_1': 'SetMem64', 'node_2': 'Buffer', 'e...
8    [{'node_1': 'UNIT_TEST_CONTEXT', 'node_2': 'UN...
9    [{'node_1': 'UNIT_TEST_CONTEXT', 'node_2': 'UN...
dtype: object
(45, 5)


In [9]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
343,unit_test_status,uintn,"50b459898e4642efa60b854f18962b0d,50b459898e464...",8,contextual proximity
344,unit_test_status,unit_test_context,"50b459898e4642efa60b854f18962b0d,50b459898e464...",8,contextual proximity
349,value,setmem64,"29121948220c429e88718249df0c5c0f,29121948220c4...",3,contextual proximity
350,value,testsafeint32touintn,"29121948220c429e88718249df0c5c0f,29121948220c4...",2,contextual proximity
351,void,efi_scsi_io_protocol,"ea30cdb45c5c492588eba156496e809b,ea30cdb45c5c4...",3,contextual proximity


In [10]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,allocatezeropool,cdb,"57ac7a8030e04e61a642964380ea95ca,5d832b56cf0c4...",contextual proximity,2
1,allocatezeropool,context,"57ac7a8030e04e61a642964380ea95ca,57ac7a8030e04...",contextual proximity,4
2,allocatezeropool,event,"5d832b56cf0c480bab92f74b5c34f43f,5d832b56cf0c4...",contextual proximity,2
3,allocatezeropool,scsilibnotify,"5d832b56cf0c480bab92f74b5c34f43f,5d832b56cf0c4...",contextual proximity,2
4,buffer,setmem64,"29121948220c429e88718249df0c5c0f,29121948220c4...",contextual proximity,3
...,...,...,...,...,...
141,unit_test_status,uintn,"50b459898e4642efa60b854f18962b0d,4c68f8d2018a4...",UNIT_TEST_STATUS is assigned the value of UINT...,16
142,unit_test_status,unit_test_context,"50b459898e4642efa60b854f18962b0d,50b459898e464...",contextual proximity,8
143,value,setmem64,"29121948220c429e88718249df0c5c0f,29121948220c4...",contextual proximity,3
144,value,testsafeint32touintn,"29121948220c429e88718249df0c5c0f,29121948220c4...",contextual proximity,2


In [11]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(41,)

In [12]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

In [13]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  5
[['a pointer to output sense data', 'hostadapterstatus', 'sensedata', 'sensedatalength', 'targetstatus', 'the length of output sense data', 'the status of host adapter'], ['allocatezeropool', 'cdb', 'commandpacket', 'context', 'createevent', 'databuffer', 'efi_event', 'efi_out_of_resources', 'event', 'executescsicommand', 'scsi io protocol to use', 'scsiio', 'scsilibnotify', 'selfevent', 'startlba', 'status', 'the context of the commandpacket'], ['buffer', 'length', 'operand', 'safeint32touintn', 'setmem64', 'testsafeint32touintn', 'value'], ['efi_scsi_io_protocol', 'efi_status', 'int64', 'uint64', 'uintn', 'unit_test_context', 'unit_test_status', 'void'], ['the length of timeout period', 'timeout']]


In [14]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,a pointer to output sense data,#5784db,1
1,hostadapterstatus,#5784db,1
2,sensedata,#5784db,1
3,sensedatalength,#5784db,1
4,targetstatus,#5784db,1
5,the length of output sense data,#5784db,1
6,the status of host adapter,#5784db,1
7,allocatezeropool,#db5f57,2
8,cdb,#db5f57,2
9,commandpacket,#db5f57,2


In [15]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [16]:
from pyvis.network import Network

graph_output_directory = "./docs/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="1200px",
    width="1200px",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
# net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# # net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
# net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

./docs/index.html
